In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import re
from moviepy.editor import *

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_holistic = mp.solutions.holistic

df = pd.DataFrame()
frames = []

In [2]:
import os
path = os.getcwd()

In [3]:
# Function to adjust video duration, frame rate, width, and height
def adjust_video_duration(video_path, output_duration, output_fps, width, height):
    video_clip = VideoFileClip(video_path, target_resolution=(height, width))
    video_clip = video_clip.without_audio()
    
    duration = video_clip.duration
    # getting only first 5 seconds
    clip = video_clip.subclip(0, duration)
    
    # applying speed effect
    final = clip.fx(vfx.speedx, duration)
    return final

In [4]:
def extract_video(video, target, sequence_id, target_width, target_height):
    global frames
    added_rows = 0
    # For webcam input:
    cap = cv2.VideoCapture(video)
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the current duration of the video
    video_duration = frame_count / fps
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert the BGR frame to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process the frame with the hand tracking model
        results = hands.process(rgb_frame)
         
        if results.multi_hand_landmarks:
            row_data = {
                "sequence_id": sequence_id,
                "target": target
            }
            detected_hands = []
            for idx, landmarks in enumerate(results.multi_hand_landmarks):
                # Iterate through detected hand landmarks
                for landmark_idx, landmark in enumerate(landmarks.landmark):
                    x, y = landmark.x * frame.shape[1], landmark.y * frame.shape[0]
                    
                    # Determine hand side based on the index of the hand in the list
                    if idx == 0:
                        hand_side = "Left"
                    else:
                        hand_side = "Right"
                    
                    row_data[f'x_{hand_side}_hand_{landmark_idx}'] =  x / target_width
                    row_data[f'y_{hand_side}_hand_{landmark_idx}'] =  y / target_height
            
                    detected_hands.append(hand_side)

                    # Draw circles on the frame
                    cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)
                    cv2.putText(frame, hand_side, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
            added_rows += 1
            frames.append(row_data)
        cv2.imshow('Hand Tracking', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        elif (added_rows == 30 and fps > 30):
            print('-' * 20)
            print("FPS: ", fps, " VIDEO DURATION: ", video_duration, "TARGET: ", target)
            print('-' * 20)
            break
    if (added_rows == 0):
        print("!! No hand detected in ", target, " with ", sequence_id)

    cap.release()
    cv2.destroyAllWindows()

In [5]:
import glob, os
import time
os.chdir("D:\Documents\Trabajo\megaproyecto_model\data/asl_lensegua")

sequence_id = 0
for file in glob.glob("*.mp4"):
    sequence_id += 1

    if (len(file.split(" ")) == 0):
        name = file.split(".")[0]
    else:
        name = " ".join(file.split(" ")[:-1])

    input_path = f'D:\Documents\Trabajo\megaproyecto_model\data/asl_lensegua/{file}'
    output_path = 'D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/adjusted_video.mp4'

    target_width = 540
    target_height = 960
    desired_fps = 30


    adjusted_video = adjust_video_duration(
        video_path=input_path,
        output_duration=1,
        output_fps=30,
        width=target_width,
        height=target_height
    )
    # Save the adjusted video to the output file
    adjusted_video.write_videofile(output_path, codec='libx264', fps=30, verbose=False, logger=None)


    extract_video(output_path, name, sequence_id, target_width, target_height)

In [6]:
df = pd.DataFrame(frames)

In [7]:
df['sequence_id'] = df['sequence_id'].astype(int)

In [8]:
len(df)

824

In [9]:
df.head()

,sequence_id,target,x_Left_hand_0,y_Left_hand_0,x_Left_hand_1,y_Left_hand_1,x_Left_hand_2,y_Left_hand_2,x_Left_hand_3,y_Left_hand_3,...,x_Right_hand_16,y_Right_hand_16,x_Right_hand_17,y_Right_hand_17,x_Right_hand_18,y_Right_hand_18,x_Right_hand_19,y_Right_hand_19,x_Right_hand_20,y_Right_hand_20
0,1,,0.651990,0.671525,0.620075,0.677205,0.601519,0.690621,0.589841,0.703892,...,0.160006,0.301258,0.148987,0.355365,0.141112,0.339069,0.136495,0.328228,0.133095,0.318162
1,1,,0.649501,0.669601,0.615327,0.675024,0.598623,0.690360,0.588844,0.704460,...,0.121969,0.314130,0.132208,0.366029,0.121715,0.351597,0.116081,0.341916,0.111406,0.332524
2,1,,0.652296,0.669732,0.618413,0.675689,0.600821,0.691006,0.590262,0.705073,...,0.166646,0.351272,0.143237,0.382855,0.133389,0.372029,0.136491,0.366892,0.143448,0.363312
3,1,,0.652009,0.670218,0.619109,0.675968,0.601392,0.691115,0.590556,0.705424,...,0.211598,0.412899,0.149574,0.410304,0.162349,0.411199,0.178736,0.414948,0.191811,0.419399
4,1,,0.653340,0.670078,0.620227,0.676265,0.602574,0.691253,0.591459,0.705469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
dir_data = "D:\Documents\Trabajo\megaproyecto_model" + "/data_lensegua.csv"
df.to_csv(dir_data, index=False)

In [11]:
validation_path = "D:\Documents\Trabajo\megaproyecto_model" + "/validation_lensegua.csv"

import random
grouped = df.groupby('target')
selected_validation_ids = []
for group_name, group_indices in grouped.groups.items():
    ids = list(set(df.loc[group_indices]['sequence_id'].values))
    if (len(ids) < 0):
        raise ("Error")
    selected_validation_ids.append(random.choice(ids))

validation_df = df[df['sequence_id'].isin(selected_validation_ids)][["sequence_id", "target"]]
validation_df.to_csv(validation_path, index=False)

In [12]:
train_path = "D:\Documents\Trabajo\megaproyecto_model" + "/train_lensegua.csv"
train_df = df[~df['sequence_id'].isin(selected_validation_ids)][["sequence_id", "target"]]
train_df.to_csv(train_path, index=False)